In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()

driver.get("https://aulavirtual.up.edu.pe/")

driver.find_element(By.ID, "agree_button").click()

username_input = driver.find_element(By.ID, "user_id")
password_input = driver.find_element(By.ID, "password")
username_input.send_keys("your_username")
password_input.send_keys("your_password")

driver.find_element(by=By.ID, value="entry-login").click()

driver.get("https://aulavirtual.up.edu.pe/ultra/course")




In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import logging
import os
from pathlib import Path
import time
from datetime import datetime


def login(driver, user, psw):
    # Create the "logs" directory if it doesn't exist
    log_dir = Path("logs")
    log_dir.mkdir(exist_ok=True)

    # Define the log file path
    log_file = log_dir / f"scraper_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )

    # Log a test message
    logging.info("This is a test log entry.")


    driver.find_element(By.ID, "agree_button").click()

    username_input = driver.find_element(By.ID, "user_id")
    password_input = driver.find_element(By.ID, "password")
    username_input.send_keys(user)
    password_input.send_keys(psw)
    driver.find_element(by=By.ID, value="entry-login").click()


In [11]:

def download_pdf(driver):
    try:
        # Wait for iframe to be present
        iframe = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "iframe"))
        )
        driver.switch_to.frame(iframe)
        
        # Wait for download button to be clickable
        download_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@title='Descargar']"))
        )
       
        download_button.click()
        
        # Wait for download to start
        time.sleep(2)  # You might need to adjust this based on download speeds
        
    except TimeoutException as e:
        logging.error(f"Timeout waiting for PDF elements")
        raise
    except Exception as e:
        logging.error(f"Error in download_pdf")
        raise
    finally:
        # Always switch back to default content, even if an error occurs
        driver.switch_to.default_content()



In [12]:

materials = driver.find_elements(
                        By.XPATH, 
                        '//*[contains(@class, "root content-list-item makeStylescontentListItem")]'
                    )

def process_material(materials):

# we want to keep opening folders (clicking) until we find an iframe or a download button
# after pressing all 'Descargar' buttons return to previous page

    for material in materials:
        try:
            
            # download direct file if found
            folder_name = material.find_element(By.XPATH, './/button').text 
            logging.info(f"Processing folder: {folder_name}")
            
            material.click()
            time.sleep(1)

            try:
                download_pdf(driver)
                driver.back()
                continue

            except:
                logging.info(f"Not direct file")

            # expand subfolders if there are
            try:
                subfolders = material.find_elements(By.XPATH, '//*[contains(@class, "makeStylesnestedItemContainer")]') 
                for subfolder in subfolders:
                    logging.info(f"Processing subfolder {subfolder.text}")
                    subfolder.click()
                

            except:
                logging.info(f"Not subfolders")
            
            
            # Find items within the expanded folder

            
            items = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((
                    By.XPATH,
                    '//*[starts-with(@class, "content-list-item makeStylescontentListItem")]'
                ))
            )
            
            # Process each item
            for item in items:
                try:
                    item.click()
                    time.sleep(1)

                    pdfs = driver.find_elements(By.XPATH,
                            '//*[starts-with(@class, "makeStylesattachmentMeta")]')
                    
                    for pdf in pdfs:
                        try:
                            pdf.click()
                            time.sleep(1)
                            download_pdf(driver)
                            pdf.click()
                        except Exception as e:
                            logging.error(f"Error processing: {str(e)}")
                            continue
                    driver.back()
                    continue
                except Exception as e:
                    logging.error(f"Error processing item in {folder_name}: {str(e)}")

                
            # Click folder again to collapse it
            material.click()
            time.sleep(1)
        except Exception as e:
            logging.error(f"Error processing material: {str(e)}")



process_material(materials)







NameError: name 'driver' is not defined

In [ ]:
def _try_download(self):
    """
    Attempts to:
    1) Download by iframe
    2) If not possible, look for 'makeStylesattachmentMeta' PDFs
    """
    try:
        # First attempt direct iframe PDF download
        _download_pdf()
        files_downloaded += 1
        return 0  # Indicate success
    except:
        logger.info('Not direct file, searching for attachments...')

    # If the direct download fails, look for attachments with 'makeStylesattachmentMeta'
    try:
        pdfs = driver.find_elements(
            By.XPATH, '//*[starts-with(@class, "makeStylesattachmentMeta")]'
        )
        for pdf in pdfs:
            try:
                pdf.click()
                time.sleep(1)
                _download_pdf()
                files_downloaded += 1
                # Click again if needed to close preview etc.
                pdf.click()
            except Exception as e:
                logger.error(f"Error processing PDF click: {e}")
        return 0
    except Exception:
        logger.info('No PDF attachments found.')
        return 1  # Indicate no attachments found

def _explore_child_elements(self,material):
        """
        Looks for <button> or <a> tags inside the current view,
        attempts pdf downloads for each.
        """
        try:
            buttons = material.find_elements(By.TAG_NAME, "button")
            for btn in buttons:
                try:
                    btn.click()
                    time.sleep(1)
                    _try_download()
                    driver.back()
                except Exception as e:
                    pass
        except:
            pass
        
        try:
            links = material.find_elements(By.TAG_NAME, "a")
            for a in links:
                try:
                    a.click()
                    time.sleep(1)
                    _try_download()
                    driver.back()
                except Exception as e:
                    pass
        except:
            pass
            
driver = webdriver.Chrome()
driver.get("https://aulavirtual.up.edu.pe/ultra/courses/_57867_1/outline")

login(driver, "d.huamanor", "123@Pato")




    

2025-01-16 23:08:55,151 - INFO - This is a test log entry.
2025-01-16 23:09:21,931 - ERROR - Timeout waiting for PDF elements: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6E8E480D5+2992373]
	(No symbol) [0x00007FF6E8ADBFD0]
	(No symbol) [0x00007FF6E897590A]
	(No symbol) [0x00007FF6E89C926E]
	(No symbol) [0x00007FF6E89C955C]
	(No symbol) [0x00007FF6E8A127D7]
	(No symbol) [0x00007FF6E89EF3AF]
	(No symbol) [0x00007FF6E8A0F584]
	(No symbol) [0x00007FF6E89EF113]
	(No symbol) [0x00007FF6E89BA918]
	(No symbol) [0x00007FF6E89BBA81]
	GetHandleVerifier [0x00007FF6E8EA6A2D+3379789]
	GetHandleVerifier [0x00007FF6E8EBC32D+3468109]
	GetHandleVerifier [0x00007FF6E8EB0043+3418211]
	GetHandleVerifier [0x00007FF6E8C3C78B+847787]
	(No symbol) [0x00007FF6E8AE757F]
	(No symbol) [0x00007FF6E8AE2FC4]
	(No symbol) [0x00007FF6E8AE315D]
	(No symbol) [0x00007FF6E8AD2979]
	BaseThreadInitThunk [0x00007FF805EEE8D7+23]
	RtlUserThreadStart [0x00007FF806E3FBCC+44]

2025-01-16 23:09:21,934 - INFO - Not direct fi

ElementClickInterceptedException: Message: element click intercepted: Element <div class="makeStylescontentTitleLine-0-2-327">...</div> is not clickable at point (351, 538). Other element would receive the click: <div class="makeStylesitemContainer-0-2-314 itemContainer makeStylesparentContentItem-0-2-350">...</div>
  (Session info: chrome=131.0.6778.265)
Stacktrace:
	GetHandleVerifier [0x00007FF6E8E480D5+2992373]
	(No symbol) [0x00007FF6E8ADBFD0]
	(No symbol) [0x00007FF6E897590A]
	(No symbol) [0x00007FF6E89D0F2E]
	(No symbol) [0x00007FF6E89CE9CC]
	(No symbol) [0x00007FF6E89CBBA6]
	(No symbol) [0x00007FF6E89CAB01]
	(No symbol) [0x00007FF6E89BCF3B]
	(No symbol) [0x00007FF6E89EF36A]
	(No symbol) [0x00007FF6E89BC596]
	(No symbol) [0x00007FF6E89EF580]
	(No symbol) [0x00007FF6E8A0F584]
	(No symbol) [0x00007FF6E89EF113]
	(No symbol) [0x00007FF6E89BA918]
	(No symbol) [0x00007FF6E89BBA81]
	GetHandleVerifier [0x00007FF6E8EA6A2D+3379789]
	GetHandleVerifier [0x00007FF6E8EBC32D+3468109]
	GetHandle